<div class="alert alert-block" style="border: 2px solid #1976D2;background-color:#E3F2FD;padding:5px;font-size:0.9em;">
본 자료는 저작권법 제25조 2항에 의해 보호를 받습니다. 본 컨텐츠 및 컨텐츠 일부 문구등을 외부에 공개, 게시하지 말아주세요.<br>
본 강의를 잘 정리하면, 데이터 분석과 데이터 과학(머신러닝, 인공지능) 모두 가능합니다!<br>
<b><a href="https://school.fun-coding.org/">잔재미코딩</a> 에서 본 강의 기반 최적화된 로드맵도 확인하실 수 있습니다</b>
</div>

### 중요도에 따른 feature 정리
> 분류 확률을 계산하는데 기여한 정도를 **피처 중요도** 라고 함 <br>
> 결과에 유의미한 영향을 주는 feature 만을 중심으로 머신러닝 기법을 적용하기도 함

### 데이터 준비

In [ ]:
import warnings
warnings.filterwarnings('ignore')

import pickle
import pandas as pd

with open('titanic_step3_feature_encoding.pickle', 'rb') as pickle_filename:
    df_onehot = pd.read_pickle(pickle_filename)
with open('titanic_step3_feature_encoding_y.pickle', 'rb') as pickle_filename: 
    y_train = pd.read_pickle(pickle_filename)
    
ntrain = 891
X_train, X_test = df_onehot[:ntrain], df_onehot[ntrain:]
X_train.head()

### XGBoost 및 LightGBM 설치

In [ ]:
!pip install lightgbm

In [ ]:
!pip install xgboost

### 라이브러리 모델 임포트

In [ ]:
import numpy as np # 각 모델에서 내부적으로 관련 라이브러리 사용 가능
import pandas as pd

from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

# knn_model 은 참고차만 사용한 것이므로(본 데이터에 좋은 성능을 확인할 수 없음), 삭제하기로 함
# from sklearn.neighbors import KNeighborsClassifier             # 1. K-Nearest Neighbor(KNN)
from sklearn.linear_model import LogisticRegression              # 2. Logistic Regression
from sklearn.svm import SVC                                      # 3. SVC
from sklearn.tree import DecisionTreeClassifier                  # 4. Decision Tree
from sklearn.ensemble import RandomForestClassifier              # 5. Random Forest
from sklearn.ensemble import ExtraTreesClassifier                # 6. Extra Tree
from sklearn.ensemble import GradientBoostingClassifier          # 7. GBM
from sklearn.naive_bayes import GaussianNB                       # 8. GaussianNB
from xgboost import XGBClassifier                                # 9. XGBoost
from lightgbm import LGBMClassifier                              # 10. LightGBM

### 머신러닝 모델링 핵심 함수 비교

#### `fit()` 함수
- **사용법**: `model.fit(X_train, y_train)`
- **목적**:
  - 모델을 데이터에 학습시키는 것
  - 모델 파라미터를 최적화하는 것
  - 실제 예측에 사용할 최종 모델을 만드는 것
- **출력**: 학습된 모델 (자기 자신을 반환)
- **언제 사용하나요?**:
  - 최종 모델을 만들어 예측에 사용할 때
  - 피처 중요도 등 학습된 모델의 속성을 확인할 때
  - 모델을 저장하거나 배포할 때
- **사용 예시**:
  ```python
  model = RandomForestClassifier()
  model.fit(X_train, y_train)
  predictions = model.predict(X_test)
  ```

#### `predict()` 함수
- **사용법**: `model.predict(X_test)`
- **목적**: 학습된 모델을 사용해 새로운 데이터에 대한 예측 수행
- **출력**: 예측 레이블 또는 값의 배열
- **언제 사용하나요?**:
  - 새로운 데이터에 대한 예측이 필요할 때
  - 모델 성능을 평가하기 위한 예측값이 필요할 때
- **사용 예시**:
  ```python
  predictions = model.predict(X_test)
  ```

#### `score()` 함수
- **사용법**: `model.score(X, y)`
- **목적**: 모델의 예측 성능을 평가
- **출력**: 각 모델별 성능 점수 (대부분의 모델은 정확도로 성능 점수를 매김) 
- **언제 사용하나요?**:
  - 학습된 모델의 성능을 빠르게 평가할 때
  - 학습 데이터와 테스트 데이터 간 성능 차이를 비교할 때
- **사용 예시**:
  ```python
  train_accuracy = model.score(X_train, y_train)
  test_accuracy = model.score(X_test, y_test)
  ```

#### `cross_val_score()` 함수
- **사용법**: `cross_val_score(model, X, y, cv=k, scoring='metric')`
- **목적**:
  - 모델 성능을 더 견고하게 검증하는 것
  - 모델이 새로운 데이터에 얼마나 잘 일반화되는지 평가하는 것
  - 모델 선택이나 하이퍼파라미터 튜닝 과정에서 과적합 방지
- **출력**: 각 폴드별 성능 점수 배열
- **언제 사용하나요?**:
  - 모델의 일반화 성능을 추정할 때
  - 여러 모델을 비교할 때
  - 하이퍼파라미터 튜닝 중 성능 평가할 때
- **사용 예시**:
  ```python
  scores = cross_val_score(model, X, y, cv=5, scoring='accuracy')
  print(f"평균 정확도: {np.mean(scores)}")
  ```

#### 종합적인 머신러닝 워크플로우
1. **탐색 단계**: `cross_val_score()`로 다양한 모델과 파라미터 평가
2. **구현 단계**: 최종 선택된 모델을 `fit()`으로 전체 훈련 데이터에 학습
3. **예측 단계**: 학습된 모델로 `predict()`를 사용해 새로운 데이터 예측
4. **평가 단계**: `score()`로 모델 성능 확인, 필요시 모델 개선

이 함수들을 적절히 조합하면 강력하고 신뢰성 있는 머신러닝 파이프라인을 구축할 수 있습니다.

### 디폴트 테스트
> 하이퍼 파라미터 튜닝을 통해 각 머신러닝 모델을 보다 최적화할 수 있지만, 우선 디폴트값으로 바로 예측 가능

In [ ]:
# knn_model 은 참고차만 사용한 것이므로(본 데이터에 좋은 성능을 확인할 수 없음), 삭제하기로 함
# knn_model = KNeighborsClassifier()
logreg_model = LogisticRegression()
svc_model = SVC()
decision_model = DecisionTreeClassifier()
random_model = RandomForestClassifier()
extra_model = ExtraTreesClassifier()
gbm_model = GradientBoostingClassifier()
nb_model = GaussianNB()
xgb_model = XGBClassifier(eval_metric='logloss')
lgbm_model = LGBMClassifier()

results = dict()

In [ ]:
alg = logreg_model
alg.fit(X_train, y_train)    
train_accuracy = alg.score(X_train, y_train) # 학습 데이터에 대한 정확도 계산
results[alg.__class__.__name__] = train_accuracy * 100 # 정확도 결과 저장 (백분율로 변환)

In [ ]:
alg = svc_model
alg.fit(X_train, y_train)    
train_accuracy = alg.score(X_train, y_train) # 학습 데이터에 대한 정확도 계산
results[alg.__class__.__name__] = train_accuracy * 100 # 정확도 결과 저장 (백분율로 변환)

In [ ]:
alg = decision_model
alg.fit(X_train, y_train)    
train_accuracy = alg.score(X_train, y_train) # 학습 데이터에 대한 정확도 계산
results[alg.__class__.__name__] = train_accuracy * 100 # 정확도 결과 저장 (백분율로 변환)

In [ ]:
alg = random_model
alg.fit(X_train, y_train)    
train_accuracy = alg.score(X_train, y_train) # 학습 데이터에 대한 정확도 계산
results[alg.__class__.__name__] = train_accuracy * 100 # 정확도 결과 저장 (백분율로 변환)

In [ ]:
alg = extra_model
alg.fit(X_train, y_train)    
train_accuracy = alg.score(X_train, y_train) # 학습 데이터에 대한 정확도 계산
results[alg.__class__.__name__] = train_accuracy * 100 # 정확도 결과 저장 (백분율로 변환)

In [ ]:
alg = gbm_model
alg.fit(X_train, y_train)    
train_accuracy = alg.score(X_train, y_train) # 학습 데이터에 대한 정확도 계산
results[alg.__class__.__name__] = train_accuracy * 100 # 정확도 결과 저장 (백분율로 변환)

In [ ]:
alg = nb_model
alg.fit(X_train, y_train)    
train_accuracy = alg.score(X_train, y_train) # 학습 데이터에 대한 정확도 계산
results[alg.__class__.__name__] = train_accuracy * 100 # 정확도 결과 저장 (백분율로 변환)

In [ ]:
alg = xgb_model
alg.fit(X_train, y_train)    
train_accuracy = alg.score(X_train, y_train) # 학습 데이터에 대한 정확도 계산
results[alg.__class__.__name__] = train_accuracy * 100 # 정확도 결과 저장 (백분율로 변환)

### 정확도 높은 순으로 정렬하기

- 사전 데이터의 value 로 정렬하는 방법 

In [ ]:
sorted(results.items(), key=lambda x: x[1], reverse=True) # reverse=True 면 높은 순서대로 정렬

### 트리 관련 모델은 중요도가 측정됨
- 트리를 결정하는 과정에서 각 feature 가 얼마나 중요한지를 수치화하며, feature_importances_ 에 해당 값을 가지고 있음
- 해당 값을 기준으로 중요도가 낮은 feature 를 걸러낼 수 있음

In [ ]:
tree_models = [
    decision_model,
    random_model,
    extra_model,
    xgb_model,    
    gbm_model
]

In [ ]:
for alg in tree_models:
    try:
        print(alg.__class__.__name__)
        print(alg.feature_importances_)        
    except:
        print(alg.__class__.__name__, "X")

#### 중요도 기반 데이터프레임 작성하기

In [ ]:
decision_model_importance = pd.DataFrame({'Feature':X_train.columns, 'decision_model':decision_model.feature_importances_})
random_model_importance = pd.DataFrame({'Feature':X_train.columns, 'random_model':random_model.feature_importances_})
extra_model_importance = pd.DataFrame({'Feature':X_train.columns, 'extra_model':extra_model.feature_importances_})
xgb_model_importance = pd.DataFrame({'Feature':X_train.columns, 'xgb_model':xgb_model.feature_importances_})
gbm_model_importance = pd.DataFrame({'Feature':X_train.columns, 'gbm_model':gbm_model.feature_importances_})

### multiple dataframe 합치기
- dataframes = [각 데이터프레임, ...]
- functools.reduce(lambda  left,right: pd.merge(left, right, on=['동일컬럼']), dataframes)
  - 다수의 DataFrame을 병합하는 파이썬 코드입니다.
    - `functools.reduce`: 리스트의 요소들을 왼쪽에서 오른쪽으로 누적하여 함수를 적용하는 함수
    - `lambda left,right`: 익명 함수로, 두 개의 DataFrame을 입력받음
    - `pd.merge(left, right, on=['동일컬럼'])`: 두 DataFrame을 '동일컬럼'을 기준으로 병합
    - `dataframes`: 병합할 DataFrame들이 담긴 리스트
  - `functools.reduce(lambda left,right: pd.merge(left, right, on=['동일컬럼']), dataframes)` 코드의 작동 방식은:
    1. `dataframes` 리스트의 첫 번째 DataFrame과 두 번째 DataFrame을 `pd.merge`를 사용해 '동일컬럼'을 기준으로 병합합니다.
    2. 그 병합 결과와 `dataframes` 리스트의 세 번째 DataFrame을 다시 병합합니다.
    3. 이런 방식으로 `dataframes` 리스트의 모든 DataFrame을 순차적으로 병합해 나갑니다.

    즉, 병합 과정이 다음과 같이 진행됩니다:
    ```
    결과1 = merge(dataframes[0], dataframes[1])
    결과2 = merge(결과1, dataframes[2])
    결과3 = merge(결과2, dataframes[3])
    ...
    최종결과 = merge(결과n-1, dataframes[n])
    ```

> 이 방식은 여러 개의 DataFrame을 동일한 기준으로 한 번에 병합할 때 매우 유용합니다.

In [ ]:
from functools import reduce
data_frames = [
    decision_model_importance,
    random_model_importance,
    extra_model_importance,
    xgb_model_importance,
    gbm_model_importance
]
importances = reduce(lambda  left,right: pd.merge(left, right, on=['Feature']), data_frames)

In [ ]:
importances.head()

#### 항목별 평균 중요도 구하기

In [ ]:
# 수치형 열만 선택 
# select_dtypes(): 특정 데이터 타입의 열만 선택하는 pandas 메서드 
# include=[np.number]: 숫자 타입의 열만 포함
# 이외에 include=['object']: 문자열, include=['datetime64']: 날짜시간, include=['bool']: 불리언 타입
numeric_columns = importances.select_dtypes(include=[np.number])

# 각 행의 수치형 열 평균 계산
# axis=1: 행 방향으로 평균 계산 (각 행의 모든 열에 대해)
# skipna=True: 결측치 무시 (기본값)
importances['avg'] = numeric_columns.mean(axis=1)

# 최종 데이터프레임 구성: 'Feature' 컬럼과 계산된 'avg' 컬럼 포함
importances = importances[['Feature', 'avg']]

#### 중요도 기반 정렬하기

In [ ]:
importances = importances.sort_values(by='avg', ascending=False)
importances.head()

### 중요도가 높은 feature 만 선택하기

In [ ]:
importances = importances[:50]

- 특정 컬럼만 선택해서, 데이터프레임 만들기

In [ ]:
train_importance = X_train[importances['Feature'].tolist()]
test_importance = X_test[importances['Feature'].tolist()]

In [ ]:
train_importance.head()

### 중요도가 높은 feature 로만 머신러닝 적용해보기

In [ ]:
logreg_model = LogisticRegression()
svc_model = SVC()
decision_model = DecisionTreeClassifier()
random_model = RandomForestClassifier()
extra_model = ExtraTreesClassifier()
gbm_model = GradientBoostingClassifier()
nb_model = GaussianNB()
xgb_model = XGBClassifier(eval_metric='logloss')
lgbm_model = LGBMClassifier()

results = dict()

In [ ]:
alg = logreg_model
alg.fit(train_importance, y_train)    
train_accuracy = alg.score(train_importance, y_train) # 학습 데이터에 대한 정확도 계산
results[alg.__class__.__name__] = train_accuracy * 100 # 정확도 결과 저장 (백분율로 변환)

In [ ]:
alg = svc_model
alg.fit(train_importance, y_train)    
train_accuracy = alg.score(train_importance, y_train) # 학습 데이터에 대한 정확도 계산
results[alg.__class__.__name__] = train_accuracy * 100 # 정확도 결과 저장 (백분율로 변환)

In [ ]:
alg = decision_model
alg.fit(train_importance, y_train)    
train_accuracy = alg.score(train_importance, y_train) # 학습 데이터에 대한 정확도 계산
results[alg.__class__.__name__] = train_accuracy * 100 # 정확도 결과 저장 (백분율로 변환)

In [ ]:
alg = random_model
alg.fit(train_importance, y_train)    
train_accuracy = alg.score(train_importance, y_train) # 학습 데이터에 대한 정확도 계산
results[alg.__class__.__name__] = train_accuracy * 100 # 정확도 결과 저장 (백분율로 변환)

In [ ]:
alg = extra_model
alg.fit(train_importance, y_train)    
train_accuracy = alg.score(train_importance, y_train) # 학습 데이터에 대한 정확도 계산
results[alg.__class__.__name__] = train_accuracy * 100 # 정확도 결과 저장 (백분율로 변환)

In [ ]:
alg = gbm_model
alg.fit(train_importance, y_train)    
train_accuracy = alg.score(train_importance, y_train) # 학습 데이터에 대한 정확도 계산
results[alg.__class__.__name__] = train_accuracy * 100 # 정확도 결과 저장 (백분율로 변환)

In [ ]:
alg = nb_model
alg.fit(train_importance, y_train)    
train_accuracy = alg.score(train_importance, y_train) # 학습 데이터에 대한 정확도 계산
results[alg.__class__.__name__] = train_accuracy * 100 # 정확도 결과 저장 (백분율로 변환)

In [ ]:
alg = xgb_model
alg.fit(train_importance, y_train)    
train_accuracy = alg.score(train_importance, y_train) # 학습 데이터에 대한 정확도 계산
results[alg.__class__.__name__] = train_accuracy * 100 # 정확도 결과 저장 (백분율로 변환)

In [ ]:
sorted(results.items(), key=lambda x: x[1], reverse=True) # reverse=True 면 높은 순서대로 정렬

<div class="alert alert-block" style="border: 2px solid #E65100;background-color:#FFF3E0;padding:10px">
<font size="4em" style="font-weight:bold;color:#BF360C;">큰그림으로 이해하기</font><br>
<font size="4em" style="color:#BF360C;">성능 개선을 위해, 피처 중요도를 계산하여, 불필요한 컬럼을 삭제하는 방법도 있음</font>
</div>

In [ ]:
import pickle
with open('titanic_step4_importance_train.pickle', 'wb') as pickle_filename:
    pickle.dump(train_importance, pickle_filename)
with open('titanic_step4_importance_test.pickle', 'wb') as pickle_filename:
    pickle.dump(test_importance, pickle_filename)
with open('titanic_step4_importance_train_y.pickle', 'wb') as pickle_filename:
    pickle.dump(y_train, pickle_filename)

<div class="alert alert-block" style="border: 2px solid #1976D2;background-color:#E3F2FD;padding:5px;font-size:0.9em;">
본 자료는 저작권법 제25조 2항에 의해 보호를 받습니다. 본 컨텐츠 및 컨텐츠 일부 문구등을 외부에 공개, 게시하지 말아주세요.<br>
본 강의를 잘 정리하면, 데이터 분석과 데이터 과학(머신러닝, 인공지능) 모두 가능합니다!<br>
<b><a href="https://school.fun-coding.org/">잔재미코딩</a> 에서 본 강의 기반 최적화된 로드맵도 확인하실 수 있습니다</b>
</div>